## [쇼핑몰 리뷰 평점 분류 경진대회](https://dacon.io/competitions/official/235938/data)



In [2]:
import pandas as pd

data_path = '../data/shopping_mall_review_scores'

#csv 형식의 training 데이터를 로드합니다.
train = pd.read_csv(f'{data_path}/train.csv')
test = pd.read_csv(f'{data_path}/test.csv')

In [3]:
train.head()

,id,reviews,target
0,0,조아요 처음구입 싸게햇어요,2
1,1,생각보다 잘 안돼요 매지 바른지 하루밖에 안됐는데ㅠㅠ 25천원가량 주고 사기 너무 ...,1
2,2,디자인은괜찮은데 상품이 금이가서 교환했는데 두번째받은상품도 까져있고 안쪽에 금이가져...,2
3,3,기전에 이 제품말고 이마트 트레이더스에서만 팔던 프리미엄 제품을 사용했었습니다. 샘...,2
4,4,튼튼하고 손목을 잘 받쳐주네요~,5


In [4]:
test.head()

,id,reviews
0,0,채소가 약간 시들어 있어요
1,1,발톱 두껍고 단단한 분들 써도 소용없어요 이 테이프 물렁거리고 힘이없어서 들어 올리...
2,2,부들부들 좋네요 입어보고 시원하면 또 살게요
3,3,이런 1. 8 골드 주라니깐 파란개 오네 회사전화걸어도 받지도 않고 머하자는거임?
4,4,검수도 없이 보내구 불량 배송비 5000원 청구하네요 완전별로 별하나도 아까워요


In [5]:
# 데이터 결측치 및 변수들의 타입 확인
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       25000 non-null  int64 
 1   reviews  25000 non-null  object
 2   target   25000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 586.1+ KB


In [6]:
# training 데이터에서 변수 추출
X = train.reviews

# training 데이터에서 변수 추출 
y = train.target 

In [7]:
from konlpy.tag import Okt
okt=Okt()  

text = "Dacon 베이직 서머에서, 시원하게 데이터 분석 하자 !" # 예시 문장을 정의합니다.

print('단어 토큰화 결과 ==>', okt.morphs(text))
print('명사 추출 결과 ==>', okt.nouns(text)) 

OpenJDK 64-Bit Server VM warning: Attempt to protect stack guard pages failed.
OpenJDK 64-Bit Server VM warning: Attempt to deallocate stack guard pages failed.


단어 토큰화 결과 ==> ['Dacon', '베이직', '서머', '에서', ',', '시원하게', '데이터', '분석', '하자', '!']
명사 추출 결과 ==> ['베이직', '서머', '데이터', '분석', '하자']


In [9]:
# okt.pos() 를 이용
print('품사 태깅 결과 ==>', okt.pos(text))  

품사 태깅 결과 ==> [('Dacon', 'Alpha'), ('베이직', 'Noun'), ('서머', 'Noun'), ('에서', 'Josa'), (',', 'Punctuation'), ('시원하게', 'Adjective'), ('데이터', 'Noun'), ('분석', 'Noun'), ('하자', 'Noun'), ('!', 'Punctuation')]


In [8]:
tokenized = [] # 데이터프레임의 한 컬럼으로 추가할 리스트
def token(dataframe):
    for sentence in dataframe['reviews']: # 전처리된 리뷰들을 하나씩 꺼내옵니다
        tokens = okt.morphs(sentence)
        tokenize = " ".join(tokens) 
        # tokens라는 리스트 안의 형태소들을 띄어쓰기로 분리된 하나의 문자열로 join시켜줍니다.
        tokenized.append(tokenize) 
        # 형태소 단위로 띄어쓰기된 문자열을 최종 리스트에 추가해줍니다
        dataframe["tokenized_stem"] = pd.DataFrame(tokenized) 
        # 리스트를 데이터프레임으로 변환해 tokenized_stem라는 컬럼명으로 추가해줍니다.
    
token(train)
token(test)

train.head() # 데이터 확인

,id,reviews,target,tokenized_stem
0,0,조아요 처음구입 싸게햇어요,2,조아요 처음 구입 싸게 햇 어 요
1,1,생각보다 잘 안돼요 매지 바른지 하루밖에 안됐는데ㅠㅠ 25천원가량 주고 사기 너무 ...,1,생각 보다 잘 안 돼요 매지 바른지 하루 밖에 안 됐는데 ㅠㅠ 25천원 가량 주고 ...
2,2,디자인은괜찮은데 상품이 금이가서 교환했는데 두번째받은상품도 까져있고 안쪽에 금이가져...,2,디자인 은 괜찮은데 상품 이 금 이 가서 교환 했는데 두번째 받은 상품 도 까져있고...
3,3,기전에 이 제품말고 이마트 트레이더스에서만 팔던 프리미엄 제품을 사용했었습니다. 샘...,2,기전 에 이 제품 말고 이마트 트 레이더스 에서만 팔던 프리미엄 제품 을 사용 했었...
4,4,튼튼하고 손목을 잘 받쳐주네요~,5,튼튼하고 손목 을 잘 받쳐주네요 ~


In [10]:
def postagging(dataframe):
    main_pos = [] # 데이터프레임의 새 컬럼이 될 리스트
    for sentence in dataframe['tokenized_stem']: # 리뷰들을 하나씩 가져옵니다
        pos = okt.pos(sentence) 
        # 형태소 분석을 진행하고 해당 리스트를 pos라는 변수로 받습니다
        main_words = [word_pos[0] for word_pos in pos if word_pos[1] in ('Adjective', #형용사
                                                                         'Noun', #명사
                                                                         'Adjective', #부사
                                                                         'Verb', #동사
                                                                         'Number',
                                                                         'KoreanParticle'
                                                                         )] 
        # 가져오고자 하는 품사에 해당하면 해당 형태소를 main_words 리스트에 추가합니다.
        main_words_str = " ".join(main_words) 
        # main_words 리스트 안의 형태소들을 띄어쓰기로 분리된 하나의 문자열로 join시켜줍니다.
        main_pos.append(main_words_str) 
        # 선택한 형태소들로 이루어진 문자열을 최종 리스트에 추가해줍니다
        dataframe["main_pos"] = pd.DataFrame(main_pos) 
        # 리스트를 데이터프레임으로 변환해 main_pos라는 컬럼명으로 추가해줍니다.

        
postagging(train)
#postagging(test) ## test셋도 똑같이 품사태깅을 적용해줍니다.
train.head() # 데이터 확인

,id,reviews,target,tokenized_stem,main_pos
0,0,조아요 처음구입 싸게햇어요,2,조아요 처음 구입 싸게 햇 어 요,조아요 처음 구입 싸게 햇 요
1,1,생각보다 잘 안돼요 매지 바른지 하루밖에 안됐는데ㅠㅠ 25천원가량 주고 사기 너무 ...,1,생각 보다 잘 안 돼요 매지 바른지 하루 밖에 안 됐는데 ㅠㅠ 25천원 가량 주고 ...,생각 보다 잘 안 돼요 매지 바른지 하루 안 됐는데 ㅠㅠ 25천원 가량 주고 사기 ...
2,2,디자인은괜찮은데 상품이 금이가서 교환했는데 두번째받은상품도 까져있고 안쪽에 금이가져...,2,디자인 은 괜찮은데 상품 이 금 이 가서 교환 했는데 두번째 받은 상품 도 까져있고...,디자인 은 괜찮은데 상품 이 금 이 가서 교환 했는데 두번째 받은 상품 도 까져있고...
3,3,기전에 이 제품말고 이마트 트레이더스에서만 팔던 프리미엄 제품을 사용했었습니다. 샘...,2,기전 에 이 제품 말고 이마트 트 레이더스 에서만 팔던 프리미엄 제품 을 사용 했었...,기전 이 제품 이마트 트 레이더스 팔던 프리미엄 제품 사용 했었습니다 샘플 써 보고...
4,4,튼튼하고 손목을 잘 받쳐주네요~,5,튼튼하고 손목 을 잘 받쳐주네요 ~,튼튼하고 손목 잘 받쳐주네요


----

In [11]:
from sklearn.model_selection import train_test_split

data = train.copy()
train, val = train_test_split(data)
train.reset_index(inplace=True) # 전처리 과정에서 데이터가 뒤섞이지 않도록 인덱스를 초기화해주었습니다.
val.reset_index(inplace=True)

In [13]:
# 크기를 확인합니다.
print( 'train 데이터 셋 모양 :', train.shape)
print( 'val 데이터 셋 모양 :', val.shape)

train 데이터 셋 모양 : (18750, 6)
val 데이터 셋 모양 : (6250, 6)


In [14]:
X_train = train.main_pos #training 데이터에서 문서 추출
y_train = train.target #training 데이터에서 라벨 추출

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer() #countvectorizer 생성
vectorizer.fit(X_train) # countvectorizer 학습
X_train_vec = vectorizer.transform(X_train) # transform

In [16]:
from sklearn.linear_model import LogisticRegression #모델 불러오기
model = LogisticRegression(max_iter=500) #객체에 모델 할당
model.fit(X_train_vec, y_train) #모델 학습

LogisticRegression(max_iter=500)

In [17]:
X_val = val.main_pos #validation 데이터에서 전처리된 문서 추출
y_val = val.target #validation 데이터에서 라벨 추출

X_val_vec = vectorizer.transform(X_val) # train셋으로 fit한 벡터라이저 이용해 transform

In [18]:
y_pred = model.predict(X_val_vec)
print(y_pred)

[5 2 5 ... 5 2 2]


In [19]:
from sklearn import metrics
print('Logistic Regression 의 예측 정확도는', round(metrics.accuracy_score(y_val, y_pred),3)) # 정확도 확인

Logistic Regression 의 예측 정확도는 0.619


In [ ]:
# CountVectorizer 적용
train_X = vectorizer.transform(train_X)
val_X = vectorizer.transform(val_X)

In [ ]:
# 변환이 잘 되었나 확인합니다.
print(train_X[10])

# inverse_transform을 사용해 원문을 확인할 수 있습니다.
vectorizer.inverse_transform(train_X[10])

In [ ]:
from sklearn.linear_model import LogisticRegression #모델 불러오기
model = LogisticRegression(max_iter=500) #객체에 모델 할당
model.fit(train_X, train_y) #모델 학습

----

In [ ]:
# 먼저 점수를 메기는 방법인 평가 지표(Metric)를 정의합니다.
import numpy as np

def ACCURACY(true, pred):   
    score = np.mean(true==pred)
    return score

In [ ]:
# val_X를 이용해 예측값을 생성합니다.
y_pred = model.predict(val_X)

In [ ]:
# 모델의 예측과 실제 정답값을 비교합니다.
score = ACCURACY(val_y, y_pred)

print(f"모델의 정확도는 {score*100:.2f}% 입니다")

----

In [ ]:
# 최종적으로 학습에 사용할 모든 리뷰를 설정해줍니다.
X = train_data.reviews
y = train_data.target

vectorizer = CountVectorizer() #countvectorizer 생성
vectorizer.fit(X) #countvectorizer 학습
X = vectorizer.transform(X) #transform

In [ ]:
model = LogisticRegression(max_iter=500) #객체에 모델 할당
model.fit(X, y) #모델 학습

In [ ]:
from sklearn.metrics import accuracy_score

#run model
y_pred = model.predict(X[0])
print('예측 라벨 : ', y_pred)
print('실제 라벨 : ', train.target[0])

In [ ]:
vectorizer.inverse_transform(X[0]) #역변환하여 첫번째 문장의 단어들 확인

In [ ]:
# 예측에 사용할 파일 읽기
test = pd.read_csv(f'{data_path}/test.csv') 

# 파일 확인
test.head() 

In [ ]:
# 문서 데이터 생성
test_X = test.reviews 

# 문서 데이터 transform 
test_X_vect = vectorizer.transform(test_X) 

In [ ]:
# test 데이터 예측
pred = model.predict(test_X_vect) 
print(pred)

----

In [ ]:
# 제출용 파일 불러오기
submission = pd.read_csv(f'{data_path}/sample_submission.csv') 

submission.head() 

In [ ]:
# 예측 값 넣어주기
submission["target"] = pred 

# 데이터가 잘 들어갔는지 확인합니다.
submission.head() 

In [ ]:
# submission을 csv 파일로 저장합니다.
# index=False란 추가적인 id를 부여할 필요가 없다는 뜻입니다. 
# 정확한 채점을 위해 꼭 index=False를 넣어주세요.
submission.to_csv(f'{data_path}/submission.csv',index=False)